In [13]:
import sklearn
import os
import pandas as pd

In [14]:
from sklearn.datasets import fetch_openml

In [15]:
mnist = fetch_openml('mnist_784', version=1)
mnist.keys()

/home/syed-khizar/anaconda3/lib/python3.11/site-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [16]:
X,y = mnist['data'] , mnist['target'] 

In [17]:
print(X.head())
print(y.head())

   pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  pixel9  \
0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
1     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

   pixel10  ...  pixel775  pixel776  pixel777  pixel778  pixel779  pixel780  \
0      0.0  ...       0.0       0.0       0.0       0.0       0.0       0.0   
1      0.0  ...       0.0       0.0       0.0       0.0       0.0       0.0   
2      0.0  ...       0.0       0.0       0.0       0.0       0.0       0.0   
3      0.0  ...       0.0       0.0       0.0       0.0       0.0       0.0   
4      0.0  ...       0.0       0.0       0.0       0.0       0.0       0.0   

   pixel781  pixel782  pixel783  pixel784  
0       0.0       

In [18]:
X.shape

(70000, 784)

In [28]:
y.shape

(70000,)

In [33]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

In [34]:
y = y.astype(np.uint8)

In [66]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

`Binary Classifier`


In [67]:
y_train_5 = y_train == 5
y_test_5 = y_test == 5

In [68]:
from sklearn.linear_model import SGDClassifier

In [69]:
sgd = SGDClassifier(random_state=42)

In [70]:
sgd.fit(X_train,y_train_5)

SGDClassifier(random_state=42)

In [71]:
digit = X.loc[0]

In [72]:
X.loc[0].shape

(784,)

In [73]:
sgd.predict([digit])

/home/syed-khizar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but SGDClassifier was fitted with feature names
  warnings.warn(


array([ True])

`Performance Measures`

In [74]:
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone

In [75]:
skfolds = StratifiedKFold(n_splits=3)

In [79]:
for train_index,test_index in skfolds.split(X_train,y_train_5):
    clone_sgd = clone(sgd)
    X_train_fold = X_train[train_index]
    y_train_fold = y_train_5[train_index]
    X_test_fold = X_test[test_index]
    y_test_fold = y_test_5[test_index]

    clone_sgd.fit(X_train_fold,y_train_fold)
    pred = clone_sgd.predict(X_test_fold)

  
   
    
    

KeyError: "None of [Index([19964, 19965, 19966, 19967, 19968, 19969, 19970, 19971, 19972, 19974,\n       ...\n       59990, 59991, 59992, 59993, 59994, 59995, 59996, 59997, 59998, 59999],\n      dtype='int64', length=40000)] are in the [columns]"

In [81]:
from sklearn.model_selection import cross_val_score
cross_val_score(sgd,X_train, y_train_5, cv=3, scoring="accuracy")

array([0.95035, 0.96035, 0.9604 ])

In [82]:
from sklearn.base import BaseEstimator

class Not5Classifier(BaseEstimator):
    def fit(self,X,y=None):
        pass
    def predict(self, X):
        return np.zeros((len(X), 1), dtype=bool)

In [84]:
never_5_clf = Not5Classifier()
cross_val_score(never_5_clf, X_train, y_train_5, cv=3, scoring="accuracy")

array([0.91125, 0.90855, 0.90915])

`Confusion Matrix `


In [85]:
from sklearn.model_selection import cross_val_predict
y_pred = cross_val_predict(sgd,X_train,y_train_5,cv=3)

In [87]:
len(y_pred)

60000

In [89]:
from sklearn.metrics import confusion_matrix

In [90]:
confusion_matrix(y_pred,y_train_5)

array([[53892,  1891],
       [  687,  3530]])

In [91]:
from sklearn.metrics import precision_score, recall_score

In [95]:
precision_score(y_train_5,y_pred)

0.8370879772350012

In [94]:
recall_score(y_train_5,y_pred)

0.6511713705958311

In [96]:
from sklearn.metrics import f1_score

In [97]:
f1_score(y_train_5,y_pred)

0.7325171197343846